# Function definition, run once

In [3]:
%matplotlib
import os
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import csv 

logFolder = r'Z:\Apparatus\DR2\Cooldown Data'
logFolder = r'/home/jaseung/jaseung_office/Labshare/Apparatus/DR2/Cooldown Data'

def loadTemperature_oneday(date):
    """
    Return two numpy array of datetime, and pressures in 'date'
    """
    basePath = os.path.join(logFolder, date)
    fileName = ['CH1 T ' + date + '.log', 'CH2 T ' + date + '.log', 'CH5 T ' + date + '.log', 'CH6 T ' + date + '.log' ]

    fullFileName = [ os.path.join(basePath, s) for s in fileName ]

    pressures_all = []
    for f in fullFileName:
        dateTimes,pressures = [], []
        fid = open(f, 'r')
        with fid:
            reader = csv.reader(fid, delimiter=",")
            for row in reader:
                dateTime = datetime.strptime(row[0]+row[1], ' %d-%m-%y%H:%M:%S')
                temperature = float(row[2])

                dateTimes.append(dateTime)
                pressures.append(temperature)

        pressures_all.append(pressures)    

    return np.asarray(dateTimes), np.transpose(np.asarray(pressures_all))

def loadPressure_oneday(date):
    """
    Return two numpy array of datetime, and pressures
    """
    basePath = os.path.join(logFolder, date)
    fileName = 'maxigauge ' + date + '.log'
    fullFileName = os.path.join(basePath, fileName)

    dateTimes,pressures = [], []
    f = open(fullFileName, 'r')
    with f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            dateTime = datetime.strptime(row[0]+row[1], '%d-%m-%y%H:%M:%S')

            pressure = [float(row[5]),float(row[11]),float(row[17]),float(row[23]),float(row[29]),float(row[35])] 

            dateTimes.append(dateTime)
            pressures.append(pressure)

    return np.asarray(dateTimes), np.asarray(pressures)

def loadFlowmeter_oneday(date):
    """
    Return two numpy array of datetime, and flowrate
    """
    basePath = os.path.join(logFolder, date)
    fileName = 'Flowmeter ' + date + '.log'
    fullFileName = os.path.join(basePath, fileName)

    dateTimes,flowrates = [], []
    f = open(fullFileName, 'r')
    with f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            dateTime = datetime.strptime(row[0]+row[1], ' %d-%m-%y%H:%M:%S')

            flowrate =float(row[2])

            dateTimes.append(dateTime)
            flowrates.append(flowrate)

    return np.asarray(dateTimes), np.asarray(flowrates)

def existingDateFolder(startDate, endDate):
    """
    Returns a list of folder names available between startDate and endDate.
    """
    start_Date = datetime.strptime(startDate, '%y-%m-%d')
    if endDate:
        end_Date = datetime.strptime(endDate, '%y-%m-%d')
    else:
        end_Date = start_Date
    dt = timedelta(days=1)
    dates = np.arange(start_Date, end_Date+dt, dt).astype(datetime)

    dateFolder = [date.strftime('%y-%m-%d') for date in dates]
    dateFolder = [ s for s in dateFolder if os.path.isdir(os.path.join(logFolder, s))]

    return dateFolder

def loadTemperature(startDate, endDate=None):
    """
    Return time and temperature numpy arrays between startDate and endDate
    """
    dateTimes_all, temperatures_all = [], []
    for i, date in enumerate(existingDateFolder(startDate, endDate)):

        dateTimes, temperatures = loadTemperature_oneday(date)

        if i ==0 :
            temperatures_all = temperatures
        else :
            temperatures_all = np.vstack((temperatures_all, temperatures))

        dateTimes_all = np.append(dateTimes_all, dateTimes)
    return dateTimes_all, temperatures_all

def loadPressure(startDate, endDate=None):
    dateTimes_all, pressures_all = [], []
    for i, date in enumerate(existingDateFolder(startDate, endDate)):

        dateTimes, pressures = loadPressure_oneday(date)

        if i ==0 :
            pressures_all = pressures
        else :
            pressures_all = np.vstack((pressures_all, pressures))

        dateTimes_all = np.append(dateTimes_all, dateTimes)
    return dateTimes_all, pressures_all

def loadFlowmeter(startDate, endDate=None):
    dateTimes_all, flowmeters_all = [], []
    for i, date in enumerate(existingDateFolder(startDate, endDate)):

        dateTimes, flowmeters = loadFlowmeter_oneday(date)

#         if i ==0 :
#             flowmeters_all = flowmeters
#         else :
#             flowmeters_all = np.vstack((flowmeters_all, flowmeters))

        dateTimes_all = np.append(dateTimes_all, dateTimes)
        flowmeters_all = np.append(flowmeters_all, flowmeters)
    return dateTimes_all, flowmeters_all

def plotTemperature(startDate, endDate=None):
    """
    Plot temperature date.
    """
    dateTimes_all, temperatures_all = loadTemperature(startDate, endDate)
    plt.figure()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_all, temperatures_all)
    plt.legend(['50K','4K','Still','MC'])
    plt.xlabel('Time')
    plt.ylabel('Temperature (K)')
    if endDate:
        plt.title('Temperature' + ' @' + startDate + ' - ' + endDate)    
    else :
        plt.title('Temperature' + ' @' + startDate)    
    plt.grid('on')

def plotPressure(startDate, endDate=None):
    
    dateTimes_all, pressures_all = loadPressure(startDate, endDate)
    plt.figure()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_all, pressures_all)
    plt.legend(['P1','P2','P3','P4','P5','P6'])
    plt.xlabel('Time')
    plt.ylabel('Pressure (mBar)')
    if endDate:
        plt.title('Pressure' + ' @' + startDate + ' - ' + endDate)    
    else :
        plt.title('Pressure' + ' @' + startDate)    
    plt.grid('on')

def plotFlowmeter(startDate, endDate=None):
    
    dateTimes_all, flowmeters_all = loadFlowmeter(startDate, endDate)
    plt.figure()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_all, flowmeters_all)
    plt.xlabel('Time')
    plt.ylabel('Flowrate (mmol/s)')
    if endDate:
        plt.title('Flowmeter' + ' @' + startDate + ' - ' + endDate)    
    else :
        plt.title('Flowmeter' + ' @' + startDate)   
    plt.grid('on')
    
def plotTempPress(startDate, endDate=None):
    dateTimes_temp_all, temperatures_all = loadTemperature(startDate, endDate)
    dateTimes_press_all, pressures_all = loadPressure(startDate, endDate)
    
    plt.subplot(211)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_temp_all, temperatures_all)
    plt.xlabel('Time')
    plt.ylabel('Temperature (K)')
    if endDate:
        plt.title('Temperature' + ' @' + startDate + ' - ' + endDate)    
    else :
        plt.title('Temperature' + ' @' + startDate)    
    plt.grid('on')
    
    plt.subplot(212)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_press_all, pressures_all)
    plt.xlabel('Time')
    plt.ylabel('Pressure (mBar)')
    if endDate:
        plt.title('Pressure' + ' @' + startDate + ' - ' + endDate)    
    else :
        plt.title('Pressure' + ' @' + startDate)  

Using matplotlib backend: TkAgg


# Plot BlueFors log

In [4]:
# Usage:
# set startDate and endDate. If only startDate is set, only data in startDate is plotted.
# Example: 
# startDate = '18-10-01'
# endDate = '18-10-10'
# plotTemperature(startDate, endDate)
# plotTemperature(startDate)


startDate = '20-04-17'
endDate = '20-04-19'

plotTemperature(startDate, endDate)
plotPressure(startDate, endDate)
plotFlowmeter(startDate, endDate)
# plotTempPress(startDate)

In [8]:
loadTemperature_oneday('19-11-13')

OSError: [Errno 22] Invalid argument: 'Z:\\Apparatus\\DR2\\Cooldown Data/19-11-13/CH1 T 19-11-13.log'